In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl

In [3]:
from subprocess import check_output
print(check_output(["ls", "../Physician_Compare/"]).decode("utf8"))

Physician_Compare_2015_Group_Public_Reporting_-_Patient_Experience.csv
Physician_Compare_2015_Group_Public_Reporting___Performance_Scores.csv
Physician_Compare_2015_Individual_EP_Public_Reporting___Performance_Scores.csv
Physician_Compare_Data_Dictionary_2015.pdf
Physician_Compare_National_Downloadable_File.csv
RPI datathon.ipynb



In [13]:
group_PE = pd.read_csv('Physician_Compare_2015_Group_Public_Reporting_-_Patient_Experience.csv')
group_score = pd.read_csv('Physician_Compare_2015_Group_Public_Reporting___Performance_Scores.csv')
indiv_score = pd.read_csv('Physician_Compare_2015_Individual_EP_Public_Reporting___Performance_Scores.csv')
national =pd.read_csv('Physician_Compare_National_Downloadable_File.csv',low_memory=False)

### Discription of 'National' datasets
national file: Gives Infomation about indivial clician, with 41 columns, important entries
* [Credential](https://en.wikipedia.org/wiki/Medical_credentials): may indicating specialities of the clinician
* Professional accepts Medicare Assignment: Assignment means that your doctor, provider, or supplier agrees (or is required by law) to accept the Medicare-approved amount as full payment for covered services.__this may indicate if the patient may be surcharged or cause complications__ 
* participated: __PQRS__-> quality rating system
    __EHR__-> use of electrical health records technology
    __MHI__-> perform well on heart health related issue
    all participation data record as Y/blank




In [21]:
print (national.columns.values)

['NPI' 'PAC ID' 'Professional Enrollment ID' 'Last Name' 'First Name'
 'Middle Name' 'Suffix' 'Gender' 'Credential' 'Medical school name'
 'Graduation year' 'Primary specialty' 'Secondary specialty 1'
 'Secondary specialty 2' 'Secondary specialty 3' 'Secondary specialty 4'
 'All secondary specialties' 'Organization legal name'
 'Group Practice PAC ID' 'Number of Group Practice members'
 'Line 1 Street Address' 'Line 2 Street Address'
 'Marker of address line 2 suppression' 'City' 'State' 'Zip Code'
 'Phone Number' 'Hospital affiliation CCN 1' 'Hospital affiliation LBN 1'
 'Hospital affiliation CCN 2' 'Hospital affiliation LBN 2'
 'Hospital affiliation CCN 3' 'Hospital affiliation LBN 3'
 'Hospital affiliation CCN 4' 'Hospital affiliation LBN 4'
 'Hospital affiliation CCN 5' 'Hospital affiliation LBN 5'
 'Professional accepts Medicare Assignment' 'Reported Quality Measures'
 'Used electronic health records'
 'Committed to heart health through the Million Hearts® initiative.']


In [128]:
# Null NA data count
national.count()

NPI                                                                  2642881
PAC ID                                                               2642881
Professional Enrollment ID                                           2642881
Last Name                                                            2642828
First Name                                                           2642853
Middle Name                                                          1997414
Suffix                                                                 52170
Gender                                                               2642881
Credential                                                            903533
Medical school name                                                  2642864
Graduation year                                                      2635864
Primary specialty                                                    2642881
Secondary specialty 1                                                 376888

## Data cleaning
- 'NPI','PAC ID' should be unique columns

1. See if there is individual with multiple ID

In [165]:
# get 7 columns of bio data
doctor_bio = national.loc[:,['NPI','PAC ID','Professional Enrollment ID','Last Name','First Name','Gender','Primary specialty','Zip Code','Hospital affiliation LBN 1']]
doctor_bio.head()

,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Gender,Primary specialty,Zip Code,Hospital affiliation LBN 1
0,1487927612,4880850486,I20120726000331,HALL,ESTHER,F,CHIROPRACTIC,956673933,NaN
1,1346282258,5395768527,I20060113000139,DAVIDSON,JOHN,M,CLINICAL SOCIAL WORKER,139015821,NaN
2,1932283124,5193762862,I20050415000143,CAGEN,STEVEN,M,CHIROPRACTIC,287129524,NaN
3,1902950462,7416123666,I20120110000522,ESPY,LEISHA,F,CHIROPRACTIC,307411348,NaN
4,1518981026,7719166586,I20110125001223,PETROSKY,DAN,M,CHIROPRACTIC,798304607,NaN


In [173]:
# repeated individual by NPI
# with unique ID the entry with multiple values must be somehow duplicated
ID_group = doctor_bio.groupby(by=['NPI','PAC ID']).count()
ID_group.head()
false_group = ID_group[ID_group['Professional Enrollment ID']!=1]
false_group.head()

,,Professional Enrollment ID,Last Name,First Name,Gender,Primary specialty,Zip Code,Hospital affiliation LBN 1
NPI,PAC ID,,,,,,,
1003000126,7517003643,4,4,4,4,4,4,4
1003000134,4284706367,11,11,11,11,11,11,11
1003000142,9931380672,9,9,9,9,9,9,9
1003000423,9133397268,2,2,2,2,2,2,2
1003000480,446348254,2,2,2,2,2,2,2


In [174]:
print('Number of duplicated entries:')
print(false_group.iloc[:,1].count())

Number of duplicated entries:
654621


In [176]:
# check random NPI in false group
check_1 = doctor_bio.loc[doctor_bio['NPI']==1003000134]
# It seems this person has been entered into the system for multiple times because of moving
print('after de-duplicates:')
check_1.drop_duplicates()


after de-duplicates:


,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Gender,Primary specialty,Zip Code,Hospital affiliation LBN 1
1104832,1003000134,4284706367,I20080707000385,CIBULL,THOMAS,M,PATHOLOGY,600261301,EVANSTON HOSPITAL
1437794,1003000134,4284706367,I20080707000385,CIBULL,THOMAS,M,PATHOLOGY,600352433,EVANSTON HOSPITAL
1473349,1003000134,4284706367,I20080707000385,CIBULL,THOMAS,M,PATHOLOGY,600761214,EVANSTON HOSPITAL
2506156,1003000134,4284706367,I20080707000385,CIBULL,THOMAS,M,PATHOLOGY,602011718,EVANSTON HOSPITAL
2638305,1003000134,4284706367,I20080707000385,CIBULL,THOMAS,M,PATHOLOGY,602014205,EVANSTON HOSPITAL


In [198]:
# check random NPI in false group
check_2 = doctor_bio.loc[doctor_bio['NPI']==1003000480]
# It seems this person has been entered into the system for multiple times because of moving
print('after de-duplicates:')
check_2.drop_duplicates()

after de-duplicates:


,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Gender,Primary specialty,Zip Code,Hospital affiliation LBN 1
1885462,1003000480,446348254,I20071113000017,ROTHCHILD,KEVIN,M,GENERAL SURGERY,800457106,UNIVERSITY OF COLORADO HOSPITAL AUTHORITY
2032104,1003000480,446348254,I20071113000017,ROTHCHILD,KEVIN,M,GENERAL SURGERY,800452545,UNIVERSITY OF COLORADO HOSPITAL AUTHORITY


### __Now, it seems that clinicians get enter into the system multiple times because of 'Zip-Code' and 'Professional Enrollment ID'__ , we can calculate the real eligible clician list by delete all duplicated entries.

In [221]:
clinician_clean = national.drop_duplicates(subset=['NPI','PAC ID'])
clinician_clean.groupby(by=['NPI','PAC ID'],as_index=False).count().head()

,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Middle Name,Suffix,Gender,Credential,Medical school name,...,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5,Professional accepts Medicare Assignment,Reported Quality Measures,Used electronic health records,Committed to heart health through the Million Hearts® initiative.
0,1003000126,7517003643,1,1,1,0,0,1,0,1,...,1,1,0,0,0,0,1,1,0,0
1,1003000134,4284706367,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,1,1,0,0
2,1003000142,9931380672,1,1,1,0,0,1,0,1,...,1,1,0,0,0,0,1,1,1,0
3,1003000407,2769663509,1,1,1,1,0,1,0,1,...,1,1,0,0,0,0,1,1,0,0
4,1003000415,6002995131,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0


In [222]:
clinician_clean[clinician_clean.loc[:,'First Name'].isna()].head()

,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Middle Name,Suffix,Gender,Credential,Medical school name,...,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5,Professional accepts Medicare Assignment,Reported Quality Measures,Used electronic health records,Committed to heart health through the Million Hearts® initiative.
302734,1518399344,6002138021,I20141202002187,VANG,NaN,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN
522644,1518288307,4082929229,I20150817001415,LIU,NaN,NaN,NaN,F,NaN,OTHER,...,200009,MAINE MEDICAL CENTER,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN
964968,1649416504,4385883222,I20130625000455,LI,NaN,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN
1009739,1033381983,4183808488,I20110412000132,MA,NaN,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN
1110143,1780918615,8426196981,I20091104000162,LI,NaN,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN


In [195]:
clinician_clean[clinician_clean.loc[:,'Last Name'].isna()].head()

,NPI,PAC ID,Professional Enrollment ID,Last Name,First Name,Middle Name,Suffix,Gender,Credential,Medical school name,...,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5,Professional accepts Medicare Assignment,Reported Quality Measures,Used electronic health records,Committed to heart health through the Million Hearts® initiative.
4419,1063454130,8628071131,I20060822000324,NaN,BENJAMIN,HANBOK,NaN,M,OD,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
28273,1689695553,4385691112,I20050405000171,NaN,SCOTT,A,NaN,M,DC,PALMER COLLEGE CHIROPRACTIC - DAVENPORT,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN
62328,1316017429,6103013941,I20101207000117,NaN,WALTER,B,IV,M,NaN,NEW YORK CHIROPRACTIC COLLEGE,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN
74993,1114025665,941246326,I20050707000585,NaN,TERRY,NaN,NaN,M,OD,INDIANA UNIVERSITY - SCHOOL OF OPTOMETRY,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN
259825,1366468308,244215952,I20111205000295,NaN,JEFFREY,A,NaN,M,NaN,HOWARD UNIVERSITY COLLEGE OF MEDICINE,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN


In [223]:
# Drop the entries without Last and First name
# clinician_clean.loc[:,'First Name'].isna()
clinician_clean = clinician_clean[clinician_clean.loc[:,'First Name'].isna() != True]
clinician_clean = clinician_clean[clinician_clean.loc[:,'Last Name'].isna() != True]

In [231]:
clinician_clean.count().head()
print ('Eligible Clinician in the system:')
clinician_clean.loc[:,'NPI'].count()

Eligible Clinician in the system:


1017770

# Conclusion
### * after data clean the eligible number of data is 101770

In [81]:
              

print('Clinicians not paticipat in PQRS:')
print(national.iloc[:,[38]][national.iloc[:,[38]]=='Y'].count()/national.iloc[:,[38]].size)
print('Clinicians not paticipat in EHR:')
print(national.iloc[:,[39]][national.iloc[:,[39]]=='Y'].count()/national.iloc[:,[39]].size)
print('Clinicians not paticipat in MHI:')
print(national.iloc[:,[40]][national.iloc[:,[40]]=='Y'].count()/national.iloc[:,[40]].size)



Clinicians not paticipat in PQRS:
Reported Quality Measures    0.708447
dtype: float64
Clinicians not paticipat in EHR:
Used electronic health records    0.26263
dtype: float64
Clinicians not paticipat in MHI:
Committed to heart health through the Million Hearts® initiative.    0.008133
dtype: float64


### Discription of ' indiv_score' dataset:
file contains measure performance rates for the 100 Individual EP 2015 Physician Quality Reporting System (PQRS) and 31 non-PQRS Qualified Clinical Data Registry (QCDR) measures reported via claims or registry that are available for public reporting. EP measure data is available for 180,723 individual EPs.
   * __Measurentment ID__:
   * __reporting Metric__: claim -> 

In [74]:
indiv_score.head()

,NPI,PAC ID,Last Name,First Name,Measure Identifier,Measure Title,Inverse Measure,Measure Performance Rate,Reporting Mechanism,Reported on PC Live Site
0,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_110_1,Preventive Care and Screening: Influenza Immun...,N,21,CLM,Y
1,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_111_1,Pneumonia Vaccination Status for Older Adults,N,28,CLM,Y
2,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_112_1,Breast Cancer Screening,N,37,CLM,Y
3,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_113_1,Colorectal Cancer Screening,N,22,CLM,Y
4,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_128_1,Preventive Care and Screening: Body Mass Index...,N,42,CLM,Y


In [84]:
# Column names
indiv_score.columns.values

array(['NPI', 'PAC ID', 'Last Name', 'First Name', 'Measure Identifier',
       'Measure Title', 'Inverse Measure', 'Measure Performance Rate',
       'Reporting Mechanism', 'Reported on PC Live Site'], dtype=object)

In [ ]:
# Check completeness


In [116]:
# types of measurement identifier : total 131 types
print('total measuremement types:')
print (indiv_score[['Measure Identifier','Measure Title']].drop_duplicates().shape)
# the data can be reported by PQRS or 
indiv_score[['Measure Identifier','Measure Title']].drop_duplicates()
# indiv_score['Measure Identifier'][indiv_score['Measure Title']=='Body Mass Index'].drop_duplicates()


total measuremement types:
(131, 2)


,Measure Identifier,Measure Title
0,PQRS_EP_110_1,Preventive Care and Screening: Influenza Immun...
1,PQRS_EP_111_1,Pneumonia Vaccination Status for Older Adults
2,PQRS_EP_112_1,Breast Cancer Screening
3,PQRS_EP_113_1,Colorectal Cancer Screening
4,PQRS_EP_128_1,Preventive Care and Screening: Body Mass Index...
5,PQRS_EP_130_1,Documentation of Current Medications in the Me...
6,PQRS_EP_181_1,Elder Maltreatment Screen and Follow-Up Plan
7,PQRS_EP_226_1,Preventive Care and Screening: Tobacco Use: Sc...
8,PQRS_EP_39_1,Screening or Therapy for Osteoporosis for Wome...
9,PQRS_EP_317_1,Preventive Care and Screening: Screening for H...


In [94]:
inv_flag = indiv_score[['Inverse Measure']]
print (inv_flag[inv_flag=='Y'].count()/inv_flag.count())

Inverse Measure    0.040961
dtype: float64


### Discription of ' Group_socre' dataset:
file contains measure performance rates for the 112 group 2015 PQRS measures reported via the Web Interface or registry that are available for public reporting. Group measure data is available for 2,371 groups.


In [100]:
# values in the file
print (group_score.head())
# group_score.columns.values

  Organization legal name or 'doing business as' name  Group PAC ID State  \
0    GLEN BURNIE PHYSICAL THERAPY AND SPORTSCARE LLC       42102592    MD   
1    GLEN BURNIE PHYSICAL THERAPY AND SPORTSCARE LLC       42102592    MD   
2    GLEN BURNIE PHYSICAL THERAPY AND SPORTSCARE LLC       42102592    MD   
3    GLEN BURNIE PHYSICAL THERAPY AND SPORTSCARE LLC       42102592    MD   
4                  HERITAGE VALLEY MEDICAL GROUP INC       42105678    PA   

  Participating in PQRS Measure Identifier  \
0                   NaN     PQRS_GRP_131_1   
1                   NaN     PQRS_GRP_154_1   
2                   NaN     PQRS_GRP_155_1   
3                   NaN     PQRS_GRP_182_1   
4                     Y     PQRS_GRP_110_1   

                                       Measure Title Inverse Measure  \
0                      Pain Assessment and Follow-Up               N   
1                             Falls: Risk Assessment               N   
2                                Falls: Plan

array(["Organization legal name or 'doing business as' name",
       'Group PAC ID', 'State', 'Participating in PQRS',
       'Measure Identifier', 'Measure Title', 'Inverse Measure',
       'Measure Performance Rate', 'Footnote', 'Reporting Mechanism',
       'Reported on PC Live Site'], dtype=object)

In [126]:
group_score[group_score.iloc[:,0]=='DUKE UNIVERSITY HOSPITAL']

,Organization legal name or 'doing business as' name,Group PAC ID,State,Participating in PQRS,Measure Identifier,Measure Title,Inverse Measure,Measure Performance Rate,Footnote,Reporting Mechanism,Reported on PC Live Site


### Discription of ' Group_PE' dataset:



In [71]:
group_PE.head()

,Organization legal name or 'doing business as' name,Group PAC ID,State,Measure Identifier,Measure Title,Measure Performance Rate,Footnote
0,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_1,"Getting timely care, appointments, and informa...",63.0,NaN
1,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_10,Between visit communication.,49.0,NaN
2,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_12,Attention to patient medication cost.,27.0,NaN
3,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_2,How well clinicians communicate.,86.0,NaN
4,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_3,Patients' rating of clinicians.,83.0,NaN


In [72]:
print (group_PE.columns.values)

["Organization legal name or 'doing business as' name" 'Group PAC ID'
 ' State' 'Measure Identifier' 'Measure Title' 'Measure Performance Rate'
 'Footnote']


In [102]:
# Types of Measurement ID and titile
group_PE[["Organization legal name or 'doing business as' name"]].drop_duplicates()

,Organization legal name or 'doing business as' name
0,HERITAGE VALLEY MEDICAL GROUP INC
8,MAYO CLINIC HEALTH SYSTEM-OWATONNA
16,UNIVERSITY OF TEXAS HEALTH SCIENCE CENTER AT S...
24,CAROLINAEAST PHYSICIANS
32,"NEWTON WELLESLEY AMBULATORY SERVICES, INC"
40,W A FOOTE MEMORIAL HOSPITAL INC
48,LOUISIANA STATE UNIVERSITY SCHOOL OF MEDICINE ...
56,"NETWORK HEALTH SYSTEM, INC."
64,EAST CAROLINA UNIVERSITY
72,SANFORD CLINIC


In [73]:
# Types of footNotes
group_PE[['Footnote']].drop_duplicates()

,Footnote
0,NaN
43,8.0


In [19]:
indiv_score.head()

,NPI,PAC ID,Last Name,First Name,Measure Identifier,Measure Title,Inverse Measure,Measure Performance Rate,Reporting Mechanism,Reported on PC Live Site
0,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_110_1,Preventive Care and Screening: Influenza Immun...,N,21,CLM,Y
1,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_111_1,Pneumonia Vaccination Status for Older Adults,N,28,CLM,Y
2,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_112_1,Breast Cancer Screening,N,37,CLM,Y
3,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_113_1,Colorectal Cancer Screening,N,22,CLM,Y
4,1508823618,42100117,GRIFFIN,DAVID,PQRS_EP_128_1,Preventive Care and Screening: Body Mass Index...,N,42,CLM,Y


In [54]:
national

,Organization legal name or 'doing business as' name,Group PAC ID,State,Measure Identifier,Measure Title,Measure Performance Rate,Footnote
0,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_1,"Getting timely care, appointments, and informa...",63.0,NaN
1,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_10,Between visit communication.,49.0,NaN
2,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_12,Attention to patient medication cost.,27.0,NaN
3,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_2,How well clinicians communicate.,86.0,NaN
4,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_3,Patients' rating of clinicians.,83.0,NaN
5,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_5,Health promotion and education.,59.0,NaN
6,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_8,Courteous and helpful office staff.,83.0,NaN
7,HERITAGE VALLEY MEDICAL GROUP INC,42105678,PA,CAHPS_GRP_9,Clinicians working together for your care.,77.0,NaN
8,MAYO CLINIC HEALTH SYSTEM-OWATONNA,42111981,MN,CAHPS_GRP_1,"Getting timely care, appointments, and informa...",42.0,NaN
9,MAYO CLINIC HEALTH SYSTEM-OWATONNA,42111981,MN,CAHPS_GRP_10,Between visit communication.,68.0,NaN
